## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

d:\py3.10\lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [6]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([784, 128]), name='weight1')
        self.b1 = tf.Variable(tf.zeros([128]), name='bias1')
        self.W2 = tf.Variable(tf.random.normal([128, 10]), name='weight2')
        self.b2 = tf.Variable(tf.zeros([10]), name='bias2')
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])  # Flatten input images
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [7]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 90.33859 ; accuracy 0.1147
epoch 1 : loss 83.186035 ; accuracy 0.119166665
epoch 2 : loss 78.2033 ; accuracy 0.121633336
epoch 3 : loss 74.36517 ; accuracy 0.1244
epoch 4 : loss 71.15582 ; accuracy 0.12631667
epoch 5 : loss 68.334274 ; accuracy 0.12823333
epoch 6 : loss 65.77736 ; accuracy 0.1304
epoch 7 : loss 63.41686 ; accuracy 0.13421667
epoch 8 : loss 61.218555 ; accuracy 0.13873333
epoch 9 : loss 59.163197 ; accuracy 0.14408334
epoch 10 : loss 57.241432 ; accuracy 0.1496
epoch 11 : loss 55.44579 ; accuracy 0.15638334
epoch 12 : loss 53.772118 ; accuracy 0.16275
epoch 13 : loss 52.213863 ; accuracy 0.16946666
epoch 14 : loss 50.762966 ; accuracy 0.17673333
epoch 15 : loss 49.412918 ; accuracy 0.18393333
epoch 16 : loss 48.15545 ; accuracy 0.19095
epoch 17 : loss 46.984447 ; accuracy 0.19758333
epoch 18 : loss 45.892025 ; accuracy 0.20475
epoch 19 : loss 44.86976 ; accuracy 0.21111667
epoch 20 : loss 43.91135 ; accuracy 0.21756667
epoch 21 : loss 43.010384 ; accuracy